In [3]:
%config IPCompleter.greedy=True

In [25]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json

In [16]:
exampleURL1 = 'https://fc2.com/en/'
response = requests.get(exampleURL1)


In [17]:
soupObject1 = BeautifulSoup(response.content,'html5lib')
soupObject1


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml"><head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>FC2 - Free Website, Analyzer, Blog, Rental Server, SEO Countermeasures, etc. -</title>
<meta content="FC2 is a portal site which brings you a pleasant web life providing blog/website/analyzer services and others. Our blog service is ranked No.2 domestically and supported by wide range of users including both beginners and heavy users. We offer as many as 30 different services." name="description"/>
<meta content="FC2 Blog, Website, Free, Analyzer, Counter, Bulletin Board, BBS, Icon, Rental Server, Shopping Mall, Shopping Cart, Mobile Phone, Mail Form, Adult" name="keywords"/>
<link href="//fc2.com/en/" rel="canonical"/>

<meta content="FC2,Inc." name="author"/>
<meta content="Copyright (c) 1999 FC2 inc. / Copyright (c)

In [7]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [18]:
singleDIV = soupObject1.find('h2',attrs={"class":"sh_heading_main_b"})
head = singleDIV.text
ddList = []
dtList = []
singleDIV = soupObject1.find('div', attrs={'class':'service_body'})
for singleDL in singleDIV.find_all('dl'):
  dtList.append( singleDL.dt.a.nextSibling.text)
  ddList.append(singleDL.dd.text)
dfFinal1 = pd.DataFrame({'Data Term':dtList, 'Data description': ddList})  
# dfFinal1.to_csv('/content/drive/My Drive/Colab/WebScraping/'+head+'.csv')

In [8]:
exampleURL2 = 'http://web.mta.info/developers/turnstile.html'
response = requests.get(exampleURL2)

In [9]:
soupObject2 =  BeautifulSoup(response.content,'html5lib')

In [12]:
singleDIV = soupObject2.find('div', attrs={'class':'span-84 last'})
for link in singleDIV.findAll('a'):
  filename = link.text
  response = requests.get('http://web.mta.info/developers/'+link.get('href'))
  soupSubObject2 = BeautifulSoup(response.content)
  with open('/content/webMtaInfo/'+filename+'.csv','w') as fd:
    fd.writelines(soupSubObject2.text)



KeyboardInterrupt: 

In [19]:
dfFinal1

,Data Term,Data description
0,Blog,Easy for beginners! Plenty of templates availa...
1,Video,Upload your favorite videos. Watch them from y...
2,Live,Broadcast a Program or enjoy a Two-way Video C...
3,SayMove!,Stream comments onto videos! It's like live co...
4,Content Market,Buy/Sell Digital Contents immediately with points
5,Free Website,"No annoying ads, large-storage (1GB)! It's free!"
6,Analyzer,Analyze visitors' movements! Choose your favor...
7,Ranking,See your own sites by ranking!
8,Counter,Lots of designs to choose from! Stop yourself ...
9,Clap,Send your cheers! Write your thanks.


Connecting with database

In [20]:
import pymongo
client = pymongo.MongoClient("localhost", 27017)
db = client.WebScraping
# db.my_collection.insert_one({"x": 10}).inserted_id

In [21]:
db.name

'WebScraping'

In [22]:
db.create_collection('fc2.com')

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'WebScraping'), 'fc2.com')

In [26]:
records = json.loads(dfFinal1.T.to_json()).values()
db.fc2.com.insert_many(records)

In [29]:
for item in db.fc2.com.find():
    print(item)

{'_id': ObjectId('5dbfab43ea7c4bd35f4ec1b2'), 'Data Term': 'Blog', 'Data description': 'Easy for beginners! Plenty of templates available!'}
{'_id': ObjectId('5dbfab43ea7c4bd35f4ec1b3'), 'Data Term': 'Video', 'Data description': 'Upload your favorite videos. Watch them from your mobile phone! Paying Members can watch in full High Definition!'}
{'_id': ObjectId('5dbfab43ea7c4bd35f4ec1b4'), 'Data Term': 'Live', 'Data description': 'Broadcast a Program or enjoy a Two-way Video Chat.'}
{'_id': ObjectId('5dbfab43ea7c4bd35f4ec1b5'), 'Data Term': 'SayMove!', 'Data description': "Stream comments onto videos! It's like live commentary! Great Fun!"}
{'_id': ObjectId('5dbfab43ea7c4bd35f4ec1b6'), 'Data Term': 'Content Market', 'Data description': 'Buy/Sell Digital Contents immediately with points'}
{'_id': ObjectId('5dbfab43ea7c4bd35f4ec1b7'), 'Data Term': 'Free Website', 'Data description': "No annoying ads, large-storage (1GB)! It's free!"}
{'_id': ObjectId('5dbfab43ea7c4bd35f4ec1b8'), 'Data Ter

In [ ]:
db.my_collection.insert_one({"x": 8}).inserted_id
db.my_collection.insert_one({"x": 11}).inserted_id
db.my_collection.find_one()


In [ ]:
for item in db.my_collection.find():
    print(item["x"])

In [ ]:
db.my_collection.create_index("x")

In [ ]:
for item in db.my_collection.find().sort("x", pymongo.ASCENDING):
    print(item["x"])

In [ ]:
[item["x"] for item in db.my_collection.find().limit(2).skip(1)]

In [ ]:
db.my_collection.insert_one(pd.DataFrame({"A":[1,2,3],"B":[4,5,6]}).to_dict()).inserted_id


In [ ]:
df = pd.DataFrame({"A":[1,2,3],"B":[4,5,6]})
records = json.loads(df.T.to_json()).values()
db.my_collection.insert_many(records)

In [ ]:
for item in db.my_collection.find().sort("A", pymongo.ASCENDING):
    print(item)

In [ ]:
#removes duplicates
db.my_collection.aggregate([
   {"$group": {"_id": {"A": "$A", "B": "$B"}}},
   {"$out": "dupCollection"}
])
db.my_collection.delete_many({})
for item in db.dupCollection.find():
    db.my_collection.insert_one(item['_id'])

In [ ]:
for item in db.my_collection.find().sort("A", pymongo.ASCENDING):
    print(item)